In [573]:
import tensorflow as tf
import numpy as np
import scipy as sp
import scipy.stats as stats
import multiprocessing as mp
import tensorflow_probability as tfp

In [578]:
N = stats.norm.cdf

def BlackScholesCall(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

In [579]:
x = tf.Variable(100.0)

with tf.GradientTape() as g:
    g.watch(x)
    C = BlackScholesCall(x,100,1.0,0,0.1)
    delta = g.gradient(C,x)

print(delta, C)

tf.Tensor(0.5199388, shape=(), dtype=float32) tf.Tensor(3.9877625, shape=(), dtype=float32)


In [558]:
def simulate_BS(it, S, sigma = 0.1):
    T = 1
    weeks = 52
    dt = T/weeks
    sum = 0

    for i in range(it):
        Sold = S
        for t in range(weeks):
            Z = np.random.normal()
            Snew = Sold*tf.math.exp((-0.5*sigma**2)*dt+sigma*tf.math.sqrt(dt)*Z)
            Sold = Snew
        if Snew - 100 > 0:
            sum = sum + (Snew - 100)
    
    return sum/float(it)

x = tf.Variable(0.1)

with tf.GradientTape() as g:
    g.watch(x)
    C = simulate_BS(1000,100,x)
    delta = g.gradient(C,x)

print(delta, C)

tf.Tensor(40.005962, shape=(), dtype=float32) tf.Tensor(3.9903426, shape=(), dtype=float32)


In [521]:
def simulate_BS(it, S0):
    sigma = 0.1

    T = 1
    weeks = 52
    dt = T/weeks

    S = tf.reshape(tf.Variable(np.repeat(S0,it), dtype = "float32"), shape = (it,1))

    for i in range(weeks):
        Z = tf.random.normal(shape=(it,1))
        temp = tf.math.multiply(tf.reshape(S[:,i], shape = (it,1)), np.exp((-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z))
        S = tf.concat([tf.reshape(S, shape = (it,i+1)), tf.reshape(temp, shape = (it,1))], axis = 1)
        
    return tf.math.reduce_sum(tf.nn.relu(S[:,i+1]-100))/it

x = tf.Variable(100.0)

with tf.GradientTape() as g:
    g.watch(x)
    C = simulate_BS(10000,x)
    delta = g.gradient(C,x)

print(delta, C)
    

None tf.Tensor(3.9710507, shape=(), dtype=float32)


In [ ]:
def simulate_BS(it, S0):
    sigma = 0.1

    T = 1
    weeks = 52
    dt = T/weeks

    S = tf.reshape(tf.Variable(np.repeat(S0,it), dtype = "float32"), shape = (it,1))

    for i in range(weeks):
        Z = tf.random.normal(shape=(it,1))
        temp = tf.math.multiply(tf.reshape(S[:,i], shape = (it,1)), np.exp((-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z))
        S = tf.concat([tf.reshape(S, shape = (it,i+1)), tf.reshape(temp, shape = (it,1))], axis = 1)
        
    return tf.math.reduce_sum(tf.nn.relu(S[:,i+1]-100))/it


with tf.GradientTape() as g:
    g.watch(x)
    C = simulate_BS(10000,x)
    delta = g.gradient(C,x)

print(delta, C)